In [28]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt

In [29]:
market = Market()
strategy = Strategy()

In [30]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()
strategy.connect()
sim = strategy.retrieve("sim")
strategy.disconnect()

In [31]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [32]:
final_sim = prices.merge(sim,on=["year","quarter","week","ticker"],how="left").dropna().reset_index()
final_sim = final_sim[["year","quarter","date","ticker","adjClose","weekly_price_regression_prediction","score"]]

In [33]:
final_sim["delta"] = (final_sim["weekly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]

In [34]:
start = final_sim["date"].min()
end = final_sim["date"].max()
date = start
seats = 3
trades = []
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in tqdm(range(70,100,10)):
        date = start
        while date <= end:
            req = float(score/100)
            if date.weekday() > 4:
                date = date + timedelta(days=1)
            try:
                todays_sim = final_sim[(final_sim["date"]==date) & (final_sim["delta"] > 0) \
                                      & (final_sim["score"] >= req)].sort_values("delta",ascending=False)
                if todays_sim.index.size < 1:
                    date = date + timedelta(days=1)
                else:
                    for seat in range(min(seats,todays_sim.index.size)):
                        try:
                            trade = todays_sim.iloc[seat]
                            ticker = trade["ticker"]
                            buy_price = trade["adjClose"]
                            exit_price = buy_price * (1+delta)
                            max_date = date + timedelta(days=7)
                            exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                (final_sim["date"] > date) & \
                                              (final_sim["date"] <= max_date) & \
                                              (final_sim["adjClose"] >= exit_price)
                                            ]
                            if exits.index.size < 1:
                                exits = final_sim[(final_sim["ticker"]==ticker) & 
                                              (final_sim["date"] > max_date)].sort_values("date",ascending=True)
                                sell_trade = exits.iloc[0]
                            else:
                                sell_trade = exits.iloc[0]
                            trade["sell_date"] = sell_trade["date"]
                            trade["sell_price"] = sell_trade["adjClose"]
                            trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
                            trade["score"] = score
                            trade["d"] = d
                            trade["seat"] = seat
                            trades.append(trade)
                            date = sell_trade["date"] + timedelta(days=1)
                        except Exception as e:
                            continue
                    date = date + timedelta(days=1)
            except Exception as e:
                date = date + timedelta(days=1)
        


  0%|                                                                                                                              | 0/3 [00:00<?, ?it/s]<ipython-input-34-50f363775b30>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade["sell_date"] = sell_trade["date"]
c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-34-50f363775b30>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

In [35]:
final_sim[(final_sim["ticker"]==ticker) &  (final_sim["date"] == beginning_date)]

,year,quarter,date,ticker,adjClose,weekly_price_regression_prediction,score,delta
404817,2020,4,2020-12-28 00:00:00+00:00,PKG,134.819516,141.727842,0.900384,0.051241


In [36]:
t = pd.DataFrame(trades)

In [37]:
a = []
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in tqdm(range(70,100,10)):
        score_trades = t[(t["score"]==score) & (t["d"]==d)]
        cash = []
        for seat in range(seats):
            initial = float(100 / seats)
            seat_trades = score_trades[score_trades["seat"]==seat]
            for delta in seat_trades["delta"]:
                initial = initial * (1+delta)
                cash.append(initial)
            analysis = {}
            analysis["score"] = score
            analysis["trades"] = seat_trades.index.size
            analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
            seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
            seat_trades["hpr"] =[x.days for x in seat_trades["hpr"]]
            analysis["hpr"] = seat_trades["hpr"].mean()
            analysis["seat"] = seat
            analysis["d"] = d
            analysis["pv"] = initial
            a.append(analysis)

  0%|                                                                                                                              | 0/3 [00:00<?, ?it/s]<ipython-input-37-3c022a3c3efc>:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
<ipython-input-37-3c022a3c3efc>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
<ipython-input-37-3c022a3c3efc>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [38]:
final = pd.DataFrame(a)

In [39]:
final

,score,trades,winrate,hpr,seat,d,pv
0,70,130,0.800000,3.415385,0,0.00,68.115359
1,70,129,0.689922,10.775194,1,0.00,38.841814
2,70,130,0.692308,19.992308,2,0.00,35.631166
3,80,132,0.803030,3.265152,0,0.00,27.134643
4,80,128,0.734375,9.789062,1,0.00,42.467867
5,80,126,0.682540,20.158730,2,0.00,27.562392
6,90,100,0.810000,12.450000,0,0.00,43.600612
7,90,93,0.795699,17.946237,1,0.00,92.502935
8,90,91,0.769231,25.846154,2,0.00,1565.280936
9,70,45,0.577778,8.533333,0,0.05,27.009122


In [40]:
hl = final.pivot_table(index=["score","d"],columns="seat",values="pv")

In [41]:
report = final.merge(hl.reset_index(),on=["score","d"]).drop(["pv","seat"],axis=1).groupby(["score","d"]).mean()
report["pv"] = [sum([row[1][x] for x in range(seats)]) for row in report.iterrows()]

In [42]:
report.sort_values("pv",ascending=False).reset_index()

,score,d,trades,winrate,hpr,0,1,2,pv
0,90,0.00,94.666667,0.791643,18.747463,43.600612,92.502935,1565.280936,1701.384484
1,90,0.05,46.000000,0.630435,39.166667,78.724680,50.542159,282.868222,412.135061
2,90,0.10,46.000000,0.630435,39.166667,78.724680,50.542159,282.868222,412.135061
3,80,0.05,50.666667,0.618824,27.394248,29.041298,36.379547,88.047903,153.468748
4,80,0.10,50.666667,0.618824,27.394248,29.041298,36.379547,88.047903,153.468748
5,70,0.00,129.666667,0.727410,11.394295,68.115359,38.841814,35.631166,142.588339
6,70,0.05,44.666667,0.559596,30.016162,27.009122,43.720757,66.494523,137.224402
7,70,0.10,44.666667,0.559596,30.016162,27.009122,43.720757,66.494523,137.224402
8,80,0.00,128.666667,0.739982,11.070981,27.134643,42.467867,27.562392,97.164902


In [43]:
benchmark = t[(t["score"]==90) & (t["d"]==0)]

In [44]:
timelines =[]
for seat in tqdm(range(seats)):
    seat_trades = benchmark[benchmark["seat"]==seat]
    initial = float(100/seats)
    timeline = []
    for row in seat_trades.iterrows():
        initial = initial * (1+row[1]["delta"])
        timeline.append(initial)
    seat_trades["pv"] = timeline
    timelines.append(seat_trades)

  0%|                                                                                                                              | 0/3 [00:00<?, ?it/s]<ipython-input-44-81cbf75264c0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["pv"] = timeline
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 136.33it/s]


In [45]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()
sp5.rename(columns={"Symbol":"ticker"},inplace=True)
benchmark = benchmark.merge(sp5,on="ticker",how="left")

In [46]:
sp5.columns

Index(['_id', 'ticker', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK',
       'Founded'],
      dtype='object')

In [47]:
# industry check
benchmark.groupby(["year","quarter","GICS Sector"]).mean().sort_values("delta",ascending=False)

adjClose  \
year quarter GICS Sector                          
2017 2       Information Technology    0.915000   
2016 3       Information Technology   46.654654   
2017 2       Materials                19.527339   
2016 3       Health Care              43.370000   
2020 4       Financials               10.989667   
2013 3       Information Technology   42.490000   
             Industrials              71.521777   
2014 1       Consumer Discretionary   56.310067   
2020 4       Materials               139.109388   
2013 3       Health Care              34.577356   
2019 4       Consumer Discretionary   34.237630   
2013 2       Information Technology    8.866756   
     4       Communication Services   21.475482   
2015 2       Health Care              82.823230   
     4       Health Care             135.625413   
2013 4       Industrials              28.273177   
2019 4       Financials               20.891885   
2015 3       Health Care              36.905267   
2019 4       Health Care              44.733559   
2016 1       Consumer Discretionary  193.788504   
2015 1       Financials                9.192370   
2016 2       Communication Services   39.391703   
2017 1       Materials                23.458737   
2013 4       Consumer Discretionary   62.429000   
2017 1       Information Technology   28.305879   
2016 1       Communication Services   17.315000   
2015 3       Communication Services   46.167039   
2016 3       Consumer Discretionary   14.116667   
2020 1       Financials               27.691835   

                                     weekly_price_regression_prediction  \
year quarter GICS Sector                                                  
2017 2       Information Technology                            1.480072   
2016 3       Information Technology                           52.224104   
2017 2       Materials                                        58.809299   
2016 3       Health Care                                      49.500583   
2020 4       Financials                                       11.966552   
2013 3       Information Technology                           44.782364   
             Industrials                                      73.706599   
2014 1       Consumer Discretionary                           73.600774   
2020 4       Materials                                       148.163150   
2013 3       Health Care                                      35.245112   
2019 4       Consumer Discretionary                           36.997063   
2013 2       Information Technology                           10.902063   
     4       Communication Services                           25.982693   
2015 2       Health Care                                      91.190619   
     4       Health Care                                     139.384611   
2013 4       Industrials                                      31.974484   
2019 4       Financials                                       23.390387   
2015 3       Health Care                                      39.499862   
2019 4       Health Care                                      48.811316   
2016 1       Consumer Discretionary                          215.757367   
2015 1       Financials                                       12.915168   
2016 2       Communication Services                           40.543996   
2017 1       Materials                                        28.327410   
2013 4       Consumer Discretionary                           80.805377   
2017 1       Information Technology                           34.303677   
2016 1       Communication Services                           19.701021   
2015 3       Communication Services                           51.070001   
2016 3       Consumer Discretionary                           19.422934   
2020 1       Financials                                       41.911994   

                                     score     delta  sell_price    d  \
year quarter GICS Sector                                  

In [48]:
benchmark.groupby(["year","quarter","ticker"]).mean().sort_values("delta",ascending=False).reset_index().merge(sp5[["ticker","Security"]],on="ticker",how="left").head(20)

,year,quarter,ticker,adjClose,weekly_price_regression_prediction,score,delta,sell_price,d,seat,CIK,Security
0,2017,2,ENPH,0.915000,1.480072,90.0,4.361275,4.688333,0.0,2.000000,1463101.0,Enphase Energy
1,2016,3,ENPH,1.180000,1.468045,90.0,0.220339,1.440000,0.0,1.000000,1463101.0,Enphase Energy
2,2017,2,CF,24.227112,58.809299,90.0,0.133018,27.505510,0.0,1.000000,1324404.0,CF Industries
3,2020,4,IVZ,10.989465,12.164653,90.0,0.121778,12.327737,0.0,2.000000,914208.0,Invesco
4,2016,3,ALXN,122.540000,139.793156,90.0,0.117431,136.930000,0.0,2.000000,899866.0,Alexion Pharmaceuticals
5,2016,3,SWKS,54.502304,61.932584,90.0,0.098946,59.895086,0.0,2.000000,4127.0,Skyworks Solutions
6,2013,3,FDX,99.365520,101.280788,90.0,0.089426,108.251340,0.0,2.000000,1048911.0,FedEx
7,2013,3,URI,52.220000,52.937114,90.0,0.085791,56.700000,0.0,1.000000,1067701.0,United Rentals
8,2016,3,AMD,5.940000,6.952568,90.0,0.072391,6.370000,0.0,1.000000,2488.0,Advanced Micro Devices
9,2013,2,SWKS,18.733416,22.429118,90.0,0.071911,20.080558,0.0,2.000000,4127.0,Skyworks Solutions
